<a href="https://colab.research.google.com/github/jorgejunior618/EstudosMachineLearning/blob/main/06_Dados_Superdimensionados_Preprocessamentos_Analiticos_e_Automatizados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from math import isnan
import pandas as pd
pd.set_option('display.max_columns', 40)
dados = pd.read_csv('https://raw.githubusercontent.com/alura-cursos/reducao-dimensionalidade/master/data-set/exames.csv')
SEED = 123143

def malBen(diag):
  return 0 if diag == 'M' else 1
def nanFill(val):
  if isnan(val): return 0
  return val
# dados.diagnostico = dados.diagnostico.map(malBen)
# dados.exame_33 = dados.exame_33.map(nanFill)
# dados

In [ ]:
exames = dados.drop(columns=['diagnostico', 'id', 'exame_33'])
diagnosticos = dados['diagnostico']

In [ ]:
'''
  Equilibrando a amostra de dados, de forma que a quantidade
  elementos com 'Diagnostico' == 1 seja proxima das de 'Diagnostico' == 0
'''

# import seaborn as sns
# from imblearn.over_sampling import SMOTE

# %matplotlib inline

# smt = SMOTE()

# exames, diagnosticos = smt.fit_resample(exames, diagnosticos)
# dados = pd.concat([exames, diagnosticos], axis=1)

# ax = sns.countplot(x='diagnostico', data=dados, hue='diagnostico')

"\n  Equilibrando a amostra de dados, de forma que a quantidade\n  elementos com 'Diagnostico' == 1 seja proxima das de 'Diagnostico' == 0\n"

# Construindo uma Visualização

Importante para interpretar a distribuição dos dados e sua relação com o diagnostico do Tumor

In [ ]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

def graficoViolino(valores, vars, inicio, fim):
  dados_plot = pd.concat([vars, valores.iloc[:,inicio:fim]], axis=1)
  dados_plot = pd.melt( dados_plot,
                        id_vars="diagnostico",
                        var_name="exames",
                        value_name="valores")

  plt.figure(figsize=(50, 15))
  plt.xticks(rotation=90)
  sns.violinplot(x="exames", y="valores", hue="diagnostico", data=dados_plot, split=True)

In [ ]:
from sklearn.preprocessing import StandardScaler

def preprocessarDadosAnalitico(data):
  '''
  Função que trata o conjunto de dados a ser usado no treinamento do modelo
  de aprendizagem de maquina, de forma a utilizar somente dados relevantes
  para a obtenção de um resultado preciso.
_____________________________________
  Remove as colunas que possuam uma variancia muito baixa, ou seja,
  que seu valor é praticamente o mesmo em todas as ocorrências
  '''
  scaler = StandardScaler()
  scaler.fit(data)
  data_pad = pd.DataFrame(scaler.transform(data), columns=data.columns)

  dataCols = data_pad.columns
  for ex in dataCols:
    variancia = data_pad[ex].var() * 100
    if data_pad[ex].max() - data_pad[ex].min() < 0.5 and variancia < 1:
      data_pad = data_pad.drop(columns=[ex])
  return data_pad

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def gerarClassificadorRandomForest(valores, resultados):
  SEED = 1234
  treino_x, teste_x, treino_y, teste_y = train_test_split(valores, resultados, stratify=resultados, test_size=0.3, random_state=SEED)

  classificador = RandomForestClassifier(n_estimators=100, random_state=SEED)
  classificador.fit(treino_x, treino_y)

  previsoes = classificador.predict(teste_x)

  accuracy = accuracy_score(np.array(teste_y), previsoes) * 100
  precision = precision_score(np.array(teste_y), previsoes, pos_label="B") * 100
  recall = recall_score(np.array(teste_y), previsoes, pos_label="B") * 100
  f1 = f1_score(np.array(teste_y), previsoes, pos_label="B") * 100

  return  (classificador,
          accuracy,
          precision,
          recall,
          f1)

def gerarClassificadorDummy(valores, resultados):
  SEED = 1234
  treino_x, teste_x, treino_y, teste_y = train_test_split(valores, resultados, stratify=resultados, test_size=0.3, random_state=SEED)

  classificador = DummyClassifier(strategy="most_frequent")

  classificador.fit(treino_x, treino_y)
  previsoes = classificador.predict(teste_x)

  accuracy = accuracy_score(np.array(teste_y), previsoes) * 100
  precision = precision_score(np.array(teste_y), previsoes, pos_label="B") * 100
  recall = recall_score(np.array(teste_y), previsoes, pos_label="B") * 100
  f1 = f1_score(np.array(teste_y), previsoes, pos_label="B") * 100

  return  (classificador,
          accuracy,
          precision,
          recall,
          f1)

In [ ]:
exames_pad = preprocessarDadosAnalitico(exames)

classificador, accuracyRandomForest, precisionRandomForest, recallRandomForest, f1RandomForest = gerarClassificadorRandomForest(exames_pad, diagnosticos)
classificadorDummy, accuracyDummy, precisionDummy, recallDummy, f1Dummy = gerarClassificadorDummy(exames_pad, diagnosticos)

print("RandomForest:",
      "\tacuracia:\t{:.2f}%".format(accuracyRandomForest),
      "\tprecisao:\t{:.2f}%".format(precisionRandomForest),
      "\trecall:\t\t{:.2f}%".format(recallRandomForest),
      "\tf1:\t\t{:.2f}%".format(f1RandomForest),
      sep="\n"
      )
print("Dummy:",
      "\tacuracia:\t{:.2f}%".format(accuracyDummy),
      "\tprecisao:\t{:.2f}%".format(precisionDummy),
      "\trecall:\t\t{:.2f}%".format(recallDummy),
      "\tf1:\t\t{:.2f}%".format(f1Dummy),
      sep="\n"
      )

RandomForest:
	acuracia:	97.66%
	precisao:	98.13%
	recall:		98.13%
	f1:		98.13%
Dummy:
	acuracia:	62.57%
	precisao:	62.57%
	recall:		100.00%
	f1:		76.98%


# Calculando a correlação entre os campos da tabela

In [ ]:
from sklearn.preprocessing import StandardScaler

def preprocessarDadosAnaliticoV2(data):
  '''
  Função que trata o conjunto de dados a ser usado no treinamento do modelo
  de aprendizagem de maquina, de forma a utilizar somente dados relevantes
  para a obtenção de um resultado preciso.

  Passo 1
  -------
  Remove as colunas que possuam uma variancia muito baixa, ou seja,
  que seu valor é praticamente o mesmo em todas as ocorrências

  Passo 2
  -------
  Remove as colunas que possuam uma alta correlação entre si
  '''
  # Normaliza o conjunto de dados para não haver grande
  # dispersão entre os valores das colunas
  scaler = StandardScaler()
  scaler.fit(data)
  data_pad = pd.DataFrame(scaler.transform(data), columns=data.columns)

  # Romevendo as colunas do conjunto de dados onde não ha grande variação
  # de forma que seus valores não interferem na classificação
  dataCols = data_pad.columns
  for ex in dataCols:
    variancia = data_pad[ex].var() * 100
    if data_pad[ex].max() - data_pad[ex].min() < 0.5 and variancia < 1:
      data_pad = data_pad.drop(columns=[ex])

  # Obtendo os subconjuntos de dados que possuam alta correlação entre si
  # de tal forma que há pouca diferença entre ter apenas um dos subconjuntos
  # contidos no conjunto de treinamento
  correlacoes = data_pad.corr()
  correlacoesOtimas = correlacoes[correlacoes > 0.99]
  variaveisCorrelacionadas = correlacoesOtimas.sum()
  correlacoesOtimas = variaveisCorrelacionadas[variaveisCorrelacionadas>1]

  remover = []
  for corr in correlacoesOtimas.keys():
    correlacoesAtual = np.array(correlacoes[correlacoes[corr] > 0.99].index)
    for i in range(len(correlacoesAtual)-1, 0, -1):
      remover.append(correlacoesAtual[i])
  remover = np.unique(remover)
  return data_pad.drop(columns=remover)

In [ ]:
exames_v2 = preprocessarDadosAnaliticoV2(exames)

classificador, accuracyRandomForest, precisionRandomForest, recallRandomForest, f1RandomForest = gerarClassificadorRandomForest(exames_v2, diagnosticos)

print("RandomForest:",
      "\tacuracia:\t{:.2f}%".format(accuracyRandomForest),
      "\tprecisao:\t{:.2f}%".format(precisionRandomForest),
      "\trecall:\t\t{:.2f}%".format(recallRandomForest),
      "\tf1:\t\t{:.2f}%".format(f1RandomForest),
      sep="\n"
      )

RandomForest:
	acuracia:	97.66%
	precisao:	97.25%
	recall:		99.07%
	f1:		98.15%


# Selecionando os K melhores atributos

Tem o objetivo de selecionar os K (valor escolhido pelo dev) atributos do conjunto de dados que melhor definem a classificação.

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import StandardScaler

def preprocessarDadosSelectKBest(data: pd.DataFrame, y) -> pd.DataFrame:
  '''
  Função que trata o conjunto de dados a ser usado no treinamento do modelo
  de aprendizagem de maquina, de forma a utilizar somente dados relevantes
  para a obtenção de um resultado preciso.

  Passo 1
  -------
  Utiliza o modulo do sklearn SelectKBest, que seleciona os K (K=7 nesta funcao)
  atributos que melhor descrevem a classificação para o conjunto de dados fornecido
  '''
  # Normaliza o conjunto de dados para não haver grande
  # dispersão entre os valores das colunas
  scaler = StandardScaler()
  scaler.fit(data)
  data_pad = pd.DataFrame(scaler.transform(data), columns=data.columns)

  # Obtendo os atributos de maior relevancia na classificão do conjunto de dados
  seletorKMelhores = SelectKBest(chi2, k=5)
  seletorKMelhores.fit(data, y)
  seletorKMelhores.set_output(transform="pandas")
  data_pad = seletorKMelhores.transform(data_pad)

  return data_pad

In [ ]:
exames_pad_v3 = preprocessarDadosSelectKBest(exames, diagnosticos)
print(exames_pad_v3.shape)

classificador, accuracyRandomForest, precisionRandomForest, recallRandomForest, f1RandomForest = gerarClassificadorRandomForest(exames_pad_v3, diagnosticos)

print("RandomForest:",
      "\tacuracia:\t{:.2f}%".format(accuracyRandomForest),
      "\tprecisao:\t{:.2f}%".format(precisionRandomForest),
      "\trecall:\t\t{:.2f}%".format(recallRandomForest),
      "\tf1:\t\t{:.2f}%".format(f1RandomForest),
      sep="\n"
      )

(569, 5)
RandomForest:
	acuracia:	97.66%
	precisao:	97.25%
	recall:		99.07%
	f1:		98.15%


In [ ]:
from sklearn.feature_selection import RFE
from sklearn.preprocessing import StandardScaler

def preprocessarDadosRFE(data: pd.DataFrame, y) -> pd.DataFrame:
  '''
  Função que trata o conjunto de dados a ser usado no treinamento do modelo
  de aprendizagem de maquina, de forma a utilizar somente dados relevantes
  para a obtenção de um resultado preciso.

  Passo 1
  -------
  Utiliza o modulo do sklearn RFE, que seleciona os N (N=5 nesta funcao)
  atributos que melhor descrevem a classificação para o conjunto de dados fornecido
  '''
  # Normaliza o conjunto de dados para não haver grande
  # dispersão entre os valores das colunas
  scaler = StandardScaler()
  scaler.fit(data)
  data_pad = pd.DataFrame(scaler.transform(data), columns=data.columns)

  classificador = gerarClassificadorRandomForest(data_pad, y)[0]
  # Obtendo os atributos de maior relevancia na classificão do conjunto de dados
  seletorRFE = RFE(estimator=classificador, n_features_to_select=5, step=1)
  seletorRFE.fit(data, y)
  seletorRFE.set_output(transform="pandas")
  data_pad = seletorRFE.transform(data_pad)

  return data_pad

In [ ]:
exames_pad_v4 = preprocessarDadosRFE(exames, diagnosticos)
classificador, accuracyRandomForest, precisionRandomForest, recallRandomForest, f1RandomForest = gerarClassificadorRandomForest(exames_pad_v4, diagnosticos)

print("RandomForest:",
      "\tacuracia:\t{:.2f}%".format(accuracyRandomForest),
      "\tprecisao:\t{:.2f}%".format(precisionRandomForest),
      "\trecall:\t\t{:.2f}%".format(recallRandomForest),
      "\tf1:\t\t{:.2f}%".format(f1RandomForest),
      sep="\n"
      )

RandomForest:
	acuracia:	95.91%
	precisao:	95.45%
	recall:		98.13%
	f1:		96.77%


In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import get_scorer_names, make_scorer

precision_score_mod = make_scorer(f1_score, pos_label="B")
def preprocessarDadosRFECV(data: pd.DataFrame, y) -> pd.DataFrame:
  '''
  Função que trata o conjunto de dados a ser usado no treinamento do modelo
  de aprendizagem de maquina, de forma a utilizar somente dados relevantes
  para a obtenção de um resultado preciso.

  Passo 1
  -------
  Utiliza o modulo do sklearn RFECV, que seleciona os N atributos que melhor
  descrevem a classificação para o conjunto de dados fornecido de acordo
  com a CrossValidation executada pelo seletor
  '''
  # Normaliza o conjunto de dados para não haver grande
  # dispersão entre os valores das colunas
  scaler = StandardScaler()
  scaler.fit(data)
  data_pad = pd.DataFrame(scaler.transform(data), columns=data.columns)

  classificador = gerarClassificadorRandomForest(data_pad, y)[0]
  seletorRFECV = RFECV(estimator=classificador, cv=5, step=1, scoring=precision_score_mod )
  # Obtendo os atributos de maior relevancia na classificão do conjunto de dados
  seletorRFECV.fit(data, y)
  seletorRFECV.set_output(transform="pandas")
  data_pad = seletorRFECV.transform(data_pad)

  return data_pad

In [ ]:
exames_pad_v5 = preprocessarDadosRFECV(exames, diagnosticos)
classificador, accuracyRandomForest, precisionRandomForest, recallRandomForest, f1RandomForest = gerarClassificadorRandomForest(exames_pad_v5, diagnosticos)

print("RandomForest:",
      "\tacuracia:\t{:.2f}%".format(accuracyRandomForest),
      "\tprecisao:\t{:.2f}%".format(precisionRandomForest),
      "\trecall:\t\t{:.2f}%".format(recallRandomForest),
      "\tf1:\t\t{:.2f}%".format(f1RandomForest),
      sep="\n"
      )

RandomForest:
	acuracia:	97.66%
	precisao:	98.13%
	recall:		98.13%
	f1:		98.13%
